-----------------------
## 1) New way of scraping  11.4.2024
-----------------------

In [1]:
from datetime import datetime
import pandas as pd
from tqdm import tqdm  
import requests 

import os
os.chdir("c:\\Users\\jirka\\Documents\\MyProjects\\Sreality")
from scraper.scraper import SrealityScraper


### My regular run

In [3]:
timestamp = "20240418_0045"
scraper = SrealityScraper()

data_prodej_byty = scraper.run_scraping(category_main_cb=1,category_type_cb=1)
df_data_prodej_byty = pd.DataFrame(data_prodej_byty)
scraper.safe_save_csv(df_data_prodej_byty, f"data_{timestamp}")

data_all = scraper.run_scraping()
df_data_all = pd.DataFrame(data_all)
scraper.safe_save_csv(df_data_all, f"data_all_{timestamp}")

2024-04-18 00:45:37
number of ALL estates (Byt, Dům, Pronájem, Prodej): 94461
we will scrape max 95 pages with 999 results each


100%|██████████| 95/95 [00:00<00:00, 368261.44it/s]


['https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&per_page=999&page=1', 'https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&per_page=999&page=2', 'https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&per_page=999&page=3', 'https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&per_page=999&page=4', 'https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&per_page=999&page=5', 'https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&per_page=999&page=6', 'https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&per_page=999&page=7', 'https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&per_page=999&page=8', 'https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&per_page=999&page=9', 'https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_t

100%|██████████| 95/95 [02:21<00:00,  1.49s/it]


16992
Saving data_20240418_0045 to C:/Users/jirka/Documents/MyProjects/Sreality/data_2024/data_20240418_0045.csv
2024-04-18 00:48:00
number of ALL estates (Byt, Dům, Pronájem, Prodej): 93927
we will scrape max 95 pages with 999 results each


100%|██████████| 95/95 [00:00<?, ?it/s]


['https://www.sreality.cz/api/cs/v2/estates?per_page=999&page=1', 'https://www.sreality.cz/api/cs/v2/estates?per_page=999&page=2', 'https://www.sreality.cz/api/cs/v2/estates?per_page=999&page=3', 'https://www.sreality.cz/api/cs/v2/estates?per_page=999&page=4', 'https://www.sreality.cz/api/cs/v2/estates?per_page=999&page=5', 'https://www.sreality.cz/api/cs/v2/estates?per_page=999&page=6', 'https://www.sreality.cz/api/cs/v2/estates?per_page=999&page=7', 'https://www.sreality.cz/api/cs/v2/estates?per_page=999&page=8', 'https://www.sreality.cz/api/cs/v2/estates?per_page=999&page=9', 'https://www.sreality.cz/api/cs/v2/estates?per_page=999&page=10', 'https://www.sreality.cz/api/cs/v2/estates?per_page=999&page=11', 'https://www.sreality.cz/api/cs/v2/estates?per_page=999&page=12', 'https://www.sreality.cz/api/cs/v2/estates?per_page=999&page=13', 'https://www.sreality.cz/api/cs/v2/estates?per_page=999&page=14', 'https://www.sreality.cz/api/cs/v2/estates?per_page=999&page=15', 'https://www.sreal

100%|██████████| 95/95 [04:41<00:00,  2.96s/it]


59940
Saving data_all_20240418_0045 to C:/Users/jirka/Documents/MyProjects/Sreality/data_2024/data_all_20240418_0045.csv


In [ ]:
# Just testing count
with requests.Session() as session:
    result =  session.request(method="GET", url="https://www.sreality.cz/api/cs/v2/estates/count").json()
result

In [ ]:
# Just testing
url_filter = f"https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&per_page=1&page=1"
with requests.Session() as session:
    result =  session.request(method="GET", url=url_filter).json()
result

## DF_ALL stats

In [ ]:
df_data_all["category_main_cb"].value_counts()

In [ ]:
df_data_all["category_type_cb"].value_counts()

In [ ]:
df_data_all["category_sub_cb"].value_counts()

## Table managment test

In [1]:
from db_managment.data_manager import DataManager
import pandas as pd

data_manager = DataManager()

In [5]:
data_manager.get_all_rows("scraped_prices")

,id,estate_id,batch_id,price,created_at
0,1,1,1,12345000,2024-04-16 21:22:23


In [ ]:
data_manager.clear_table("scraped_prices")

In [ ]:
data_manager.drop_table("scraped_prices")

In [ ]:
data_manager.create_table("scraped_prices")

In [ ]:
upload = pd.DataFrame(data = [["2024-15-04 12:00:01"]],
                      columns=["timestamp"])
data_manager.insert_new_batch(upload)

In [2]:

upload = pd.DataFrame(data = [["2109527372",
                              "Prodej bytu 3+1 69 m²",
                              "1",
                              "1",
                              "7",
                              "byt",
                              "prodej",
                              "3+1",
                              "69.0",
                              "brno-zabovresky-lucni",
                              "jihomoravský kraj",
                              "Brno",
                              "49.208683",
                              "16.58177716",
                              "2024-04-13 10:34:53",
                              "12345000",
                              "2024-04-16 21:22:23"
                              ]],
                      columns=["code", "name", 
                               "category_main_cb", "category_type_cb", "category_sub_cb",
                               "type_of_building", "type_of_deal", "type_of_rooms", 
                               "size_meters", "locality",
                               "region", "district", "latitude",
                               "longitude", "timestamp", "price", "created_at"])
data_manager.insert_new_estate(upload)

In [4]:

#TODO: how about unit test for this?
#? should fail predictably as it breakes foreign key constraints i made:)
#upload = pd.DataFrame(data = [["911", "4242", "12345000"]],
#                      columns=["estate_id", "batch_id", "price"])

upload = pd.DataFrame(data = [["1", "1", "12345000", "2024-04-16 21:22:23"]],
                      columns=["estate_id", "batch_id", "price", "created_at"])
data_manager.insert_new_price(upload)

## Processing new offers

In [ ]:
from db_managment.data_manager import DataManager
import pandas as pd

data_manager = DataManager()

In [5]:
df_new = pd.read_csv("data_2024/data_20240418_0045.csv", sep=";", encoding="utf-8")

print(len(df_new))
display(df_new.tail(2))

16992


,Unnamed: 0,code,name,category_main_cb,category_type_cb,category_sub_cb,rooms,size_meters,locality,latitude,longitude,price,timestamp
16990,16990,1653154140,Prodej bytu 4+kk 140 m²,1,1,8,4+kk,140.0,karlovy-vary-karlovy-vary-sadova,50.210766,12.892186,11963000,2024-04-18 00:45:37
16991,16991,1673015644,Prodej bytu atypické 92 m² (Podkrovní),1,1,16,atypické,92.0,stachy-stachy-zadov,49.062504,13.649163,8983000,2024-04-18 00:45:37


In [7]:
new_estates = data_manager.filter_new_estates(df_new)

print(len(new_estates))
display(new_estates.tail(2))

28659


,Unnamed: 0,hash_id,name,price,latitude,longitude,category_main_cb,category_type_cb,category_sub_cb,type_of_flat,size_m
28657,28657,1653154140,Prodej bytu 4+kk 74 m²,6362000,50.217776,12.885176,1,1,8,4+kk,74.0
28658,28658,1673015644,Prodej bytu atypické 49 m² (Podkrovní),4777000,49.069514,13.642153,1,1,16,atypické,49.0


In [12]:
print(new_estates.loc[0, "category_main_cb"])
print(data_manager.translate_type_of_building(new_estates.loc[0, "category_main_cb"]))

print(new_estates.loc[0, "category_type_cb"])
print(data_manager.translate_type_of_deal(new_estates.loc[0, "category_main_cb"]))

print(new_estates.loc[0, "category_sub_cb"])
print(data_manager.translate_type_of_rooms(new_estates.loc[0, "category_main_cb"]))

1
byt
1
prodej
4
N/A


worfklow new price, pokud neexistuje Estate, resp batch
- load csv s offers
- filter_new_estates - najde mi ty, jejichž code ještě není v tabulce estate_details
- přeloží to pro ně typ budovy, typ  dealu, počet pokojů - z dictionary
- před zápisem do DB vygeneruje timestamp 
- insertuje se new_batch - pokud taky neexistuje !!
- jinak se může insertovat new new_estate
- a pak klidně rovnou i new price, navázané na tuto batch a price